In [ ]:
import os
import numpy as np
from caret_analyze import Architecture, Application, check_procedure, Lttng, LttngEventFilter
from caret_analyze.plot import Plot, message_flow, callback_graph, chain_latency, callback_sched
from bokeh.plotting import output_notebook, figure, show
import pandas as pd
import pandas_bokeh
import matplotlib.pyplot as plt
import IPython
from IPython.display import Image, display_svg
output_notebook()

In [ ]:
tracing_log_path = os.environ['HOME'] + '/.ros/tracing/caret_sample'
architecture_raw_path = 'raw_architecture.yaml'

context_type = 'use_latest_message'
# context_type = 'callback_chain'

architecture_fixed_path = 'sample_straight.yaml'
from make_paths_straight import make_paths

# architecture_fixed_path = 'sample_huge_latency.yaml'
# from make_paths_straight import make_paths

# architecture_fixed_path = 'sample_store.yaml'
# from make_paths_straight import make_paths

# architecture_fixed_path = 'sample_store_timer_1.yaml' if context_type == 'use_latest_message' else 'sample_store_timer_1_callbackchain.yaml'
# from make_paths_straight import make_paths

# architecture_fixed_path = 'sample_store_timer_2.yaml' if context_type == 'use_latest_message' else 'sample_store_timer_2_callbackchain.yaml'
# from make_paths_straight import make_paths

# architecture_fixed_path = 'sample_sub3pub1.yaml'
# from make_paths_sub3pub1 import make_paths

# architecture_fixed_path = 'sample_sub1pubtimer3.yaml' if context_type == 'use_latest_message' else 'sample_sub1pubtimer3_callbackchain.yaml'
# from make_paths_sub1pubtimer3 import make_paths

# architecture_fixed_path = 'sample_feedback.yaml' if context_type == 'use_latest_message' else 'sample_feedback_callbackchain.yaml'
# from make_paths_straight import make_paths

In [ ]:
lttng = Lttng(tracing_log_path, force_conversion=True)
arch = Architecture('lttng', tracing_log_path)
arch.export(architecture_raw_path, force=True)

In [ ]:
path_name_list = make_paths(architecture_raw_path, "temp.yaml")

# then, copy "temp.yaml" to architecture_fixed_path, and modify it if needed
#  - modify context_type (UNDEFINED -> use_latest_message), or
#  - modify context_type (UNDEFINED -> callback_chain) and set variable_passings

In [ ]:
arch = Architecture('yaml', architecture_fixed_path)
for target_pathname in path_name_list:
    path = arch.get_path(target_pathname)
    print(path.verify())

In [ ]:
lttng = Lttng(tracing_log_path, force_conversion=True, event_filters=[LttngEventFilter.duration_filter(0.3, 1.0)])
app = Application(arch, lttng)

path_list = []
for path_name in path_name_list:
    path_list.append(app.get_path(path_name))

In [ ]:
# Message Flow
for target_path in path_list:
    message_flow(target_path, granularity='node', treat_drop_as_delay=False)
    message_flow(target_path, granularity='node', treat_drop_as_delay=True)

In [ ]:
# Chain Latency
for path in path_list:
    print(f'*** {path.path_name} ***')
    graph = chain_latency(path, granularity='node', treat_drop_as_delay=False)
    display_svg(graph)
    # display_svg(chain_latency(path, granularity='node', treat_drop_as_delay=False))
    # graph.view(path.path_name)

In [ ]:
# Average and Timechart for each path
for name, path in zip(path_name_list, path_list):
    abs_ts, latency_ns = path.to_timeseries(remove_dropped=True, treat_drop_as_delay=False)
    # abs_ts, latency_ns = path.to_timeseries(remove_dropped=False, treat_drop_as_delay=True)
    latency_ms = latency_ns * 10**-6 # latensy as milliseconds
    
    stats_latency = np.array([item for item in latency_ms if isinstance(item, float)])
    print(f"Path: {name}")
    print(f"mean: {stats_latency.mean()} msec, std: {stats_latency.std()}")
    print(f"min : {stats_latency.min()} msec, max : {stats_latency.max()}")
    
    rel_ts = (abs_ts - abs_ts[0]) * 10**-9 # timetamp as seconds
    p = figure()
    p.line(rel_ts, latency_ms)
    p.y_range.start = 0
    show(p)

    bins, hist = path.to_histogram(treat_drop_as_delay=False)
    p = figure()
    hist = hist*10**-6 # nanoseconds to milliseconds
    p.step(hist[1:], bins)
    show(p)
